In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import pyvista as pv
import os
import utils
from stpyvista import stpyvista
from datetime import datetime
import importlib

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\stpyvista\__init__.py:18: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("vtk", sizing_mode="stretch_both")


In [3]:
df = pd.concat([pd.read_csv('hka-aqm-am/' + f.removeprefix('._'), skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])
df.shape


(608036, 18)

In [7]:
importlib.reload(utils)
performance_df = pd.read_csv('model_performances.csv')
d_model=256
nhead=8
num_layers=5 
dropout_pe=0.15
dropout_encoder=0.15
batch_size=1024
learning_rate=0.00031
epochs=25
for aggregation_level in ['quarter_hour', 'hour', 'half_hour']:
    for y_feature in ['CO2', 'VOC', 'hum', 'tmp', 'vis']:
        
        if y_feature == 'hum':
            # hum we are overfitting with the same parameters. Decreasing num_layers by 2
            num_layers -= 2
        elif y_feature == 'tmp':
            # tmp we are overfitting a little bit with the same parameters. Increasing dropout probability by 0.05 and decreasing num_layers by 1
            num_layers -= 1
            dropout_pe += 0.05
            dropout_encoder += 0.05
        
        model, scaler, rmse, mae, me, mape, train_loss, val_loss, n_features = utils.create_multivariate_transformer_model_for_feature(df, d_model=d_model, nhead=nhead, num_layers=num_layers, dropout_pe=dropout_pe, dropout_encoder=dropout_encoder, batch_size=batch_size, learning_rate=learning_rate, epochs=epochs, y_feature=y_feature, aggregation_level=aggregation_level)
        performance_df = performance_df.append({'model_name': 'multivariate_transformer','aggregation_level': aggregation_level, 'y_feature': y_feature, 'n_features': n_features, 'rmse': rmse, 'mae': mae, 'me': me, 'mape': mape, 'd_model': d_model, 'nhead': nhead, 'num_layers': num_layers, 'dropout_pe': dropout_pe,'dropout_encoder': dropout_encoder, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs, 'train_loss': train_loss, 'val_loss': val_loss, 'note': '2 decoder Linear layers with LeakyReLu, shuffled training data'}, ignore_index=True)
        performance_df.to_csv('model_performances.csv', index=False)
        
        if y_feature == 'hum':
            # reverting changes
            num_layers += 2
        if y_feature == 'tmp':
            # reverting changes
            num_layers += 1
            dropout_pe -= 0.05
            dropout_encoder -= 0.05


training data cutoff:  2023-07-15 02:45:00


c:\Studium\Semester_6\IoT_Project\utils.py:1036: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_scale] = scaler.fit_transform(df_train[columns_to_scale])
c:\Studium\Semester_6\IoT_Project\utils.py:1037: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[columns_to_scale] = scaler.transform(df_test[columns_to_scale])
c:\Studium\Semester_6\IoT_Project\utils.py:1041: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the document

device_id              int8
tmp                 float64
hum                 float64
CO2                 float64
VOC                 float64
vis                 float64
IR                  float64
WIFI                float64
BLE                 float64
rssi                float64
channel_rssi        float64
channel_index       float64
spreading_factor    float64
bandwidth           float64
f_cnt               float64
isHoliday           float64
isExamTime          float64
weekday             float64
month               float64
hour_sin            float64
semester_SS23       float64
semester_WS22/23    float64
semester_WS23/24    float64
group                 int32
dtype: object
Training data shape: torch.Size([244176, 20, 22]) torch.Size([244176]) torch.Size([244176, 1])
Testing data shape: torch.Size([62745, 20, 22]) torch.Size([62745]) torch.Size([62745, 1])
Shuffled Training data shape: torch.Size([245536, 20, 22]) torch.Size([245536]) torch.Size([245536, 1])
Shuffled Testing data sh

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


          actual   predicted
0     484.000000  481.173129
1     444.000001  437.261614
2     398.999996  418.653904
3     392.999999  412.243304
4     431.800001  438.165256
...          ...         ...
1019  860.000011  646.743423
1020  509.999999  483.710021
1021  410.999999  430.045980
1022  436.000002  435.739474
1023  351.999999  402.860421

[1024 rows x 2 columns]
Epoch 1/25, Training Loss: 0.2807
Epoch 1/25, Validation Loss: 0.2417
          actual   predicted
0     484.000000  495.082365
1     444.000001  434.960756
2     398.999996  429.274586
3     392.999999  408.102637
4     431.800001  448.479905
...          ...         ...
1019  860.000011  620.003233
1020  509.999999  503.824992
1021  410.999999  421.133949
1022  436.000002  433.584959
1023  351.999999  386.292366

[1024 rows x 2 columns]
Epoch 2/25, Training Loss: 0.1781
Epoch 2/25, Validation Loss: 0.1290
          actual   predicted
0     484.000000  493.263653
1     444.000001  418.765236
2     398.999996  391.76395

...

          actual   predicted
0       5.499997  252.377539
1     218.250000  275.077255
2     283.124996  404.700862
3       7.333340   41.998723
4      37.749993   95.893761
...          ...         ...
1019   45.749995   72.671586
1020   30.249998   91.291315
1021    5.499997   96.258704
1022    4.499999  100.029531
1023  129.250002   76.347963

[1024 rows x 2 columns]
Epoch 1/25, Training Loss: 0.8585
Epoch 1/25, Validation Loss: 0.6413
          actual   predicted
0       5.499997  668.951393
1     218.250000  834.289689
2     283.124996  698.404814
3       7.333340   26.084571
4      37.749993  237.022261
...          ...         ...
1019   45.749995  164.030240
1020   30.249998  400.322214
1021    5.499997  348.923617
1022    4.499999  441.728746
1023  129.250002  219.181549

[1024 rows x 2 columns]
Epoch 2/25, Training Loss: 0.7065
Epoch 2/25, Validation Loss: 0.6484
          actual   predicted
0       5.499997   81.021378
1     218.250000  133.212880
2     283.124996  104.17212

c:\Studium\Semester_6\IoT_Project\utils.py:1036: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_scale] = scaler.fit_transform(df_train[columns_to_scale])
c:\Studium\Semester_6\IoT_Project\utils.py:1037: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[columns_to_scale] = scaler.transform(df_test[columns_to_scale])
c:\Studium\Semester_6\IoT_Project\utils.py:1041: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the document

Training data shape: torch.Size([152512, 20, 22]) torch.Size([152512]) torch.Size([152512, 1])
Testing data shape: torch.Size([38893, 20, 22]) torch.Size([38893]) torch.Size([38893, 1])
Shuffled Training data shape: torch.Size([153124, 20, 22]) torch.Size([153124]) torch.Size([153124, 1])
Shuffled Testing data shape: torch.Size([38281, 20, 22]) torch.Size([38281]) torch.Size([38281, 1])
23


c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


           actual    predicted
0      500.000000   482.028006
1      446.000001   418.528078
2      554.500002   554.296513
3      544.000002   560.509294
4     1438.249999  1414.383774
...           ...          ...
1019   409.999996   408.464830
1020   419.500001   412.822871
1021   403.999996   411.939183
1022   458.666666   436.479332
1023   379.500000   401.357136

[1024 rows x 2 columns]
Epoch 1/25, Training Loss: 0.3872
Epoch 1/25, Validation Loss: 0.1963
           actual    predicted
0      500.000000   486.811213
1      446.000001   435.807489
2      554.500002   553.677169
3      544.000002   548.962401
4     1438.249999  1330.488468
...           ...          ...
1019   409.999996   429.622614
1020   419.500001   432.868699
1021   403.999996   433.799065
1022   458.666666   450.664642
1023   379.500000   421.293568

[1024 rows x 2 columns]
Epoch 2/25, Training Loss: 0.2216
Epoch 2/25, Validation Loss: 0.1827
           actual    predicted
0      500.000000   497.663784
1   

In [26]:
all_model_files = [f for f in os.listdir('models/')]
all_model_files_no_version = [f.rsplit('_', 1)[0] for f in all_model_files]
all_model_files_no_version_unique = list(set(all_model_files_no_version))
all_model_files_no_version_unique
for model_file in all_model_files_no_version_unique:
    model_files = [f for f in all_model_files if f.startswith(model_file)]
    model_files.sort()
    latest_model_file = model_files[-1]
    print(latest_model_file)
    # delete all other versions
    for model_file_to_delete in model_files[:-1]:
        print('deleting ' + model_file_to_delete)
        os.remove('models/' + model_file_to_delete)
    # rename latest model file to v1
    file_ending = latest_model_file.rsplit('.', 1)[1]
    latest_model_file_rename = latest_model_file.rsplit('_', 1)[0] + '_v1' + '.' + file_ending
    if latest_model_file != latest_model_file_rename:
        print('renaming ' + latest_model_file + ' to ' + latest_model_file_rename)
        os.rename('models/' + latest_model_file, 'models/' + model_file + '_v1')

transformer_multivariate_quarter_hour_vis_columns_v1.csv
transformer_multivariate_quarter_hour_tmp_model_v1.pth
transformer_multivariate_half_hour_tmp_columns_v1.csv
transformer_multivariate_hour_vis_columns_v1.csv
transformer_model_tmp_v1.pth
transformer_multivariate_hour_hum_model_v1.pth
.DS_v1
renaming .DS_v1 to .DS_v1.DS_v1
transformer_multivariate_hour_vis_scaler_v1.pth
transformer_multivariate_hour_vis_model_v1.pth
transformer_multivariate_hour_tmp_columns_v1.csv
transformer_scaler_tmp_v1.pth
transformer_multivariate_half_hour_CO2_model_v1.pth
transformer_multivariate_half_hour_hum_scaler_v1.pth
transformer_multivariate_quarter_hour_CO2_model_v1.pth
transformer_multivariate_quarter_hour_hum_columns_v1.csv
transformer_multivariate_hour_hum_columns_v1.csv
transformer_multivariate_quarter_hour_CO2_scaler_v1.pth
transformer_multivariate_quarter_hour_vis_model_v1.pth
transformer_multivariate_hour_tmp_scaler_v1.pth
transformer_multivariate_half_hour_VOC_columns_v1.csv
transformer_multi

In [7]:
importlib.reload(utils)
model, scaler, rmse, mae, mape = utils.create_multivariate_transformer_model_for_feature(df, d_model=128, nhead=8, num_layers=4, dropout=0.1, batch_size=128, learning_rate=0.00025, epochs=2)

training data cutoff:  2023-07-15 02:45:00


/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:868: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_count = x[0].shape[1]
/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:869: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  print("feature count: ", feature_count)
/Users/timehmann/Library/Mob

tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday                     float64
month                       float64
hour_sin                    float64
semester_SS23               float64
semester_WS22/23            float64
semester_WS23/24            float64
group                       float64
device_id_hka-aqm-am001     float64
device_id_hka-aqm-am002     float64
device_id_hka-aqm-am003a    float64
device_id_hka-aqm-am003b    float64
device_id_hka-aqm-am004     

TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
importlib.reload(utils)
model, scaler = utils.create_multivariate_model_for_feature(df, aggregation_level='half_hour')

Training data shape: torch.Size([150050, 47]) torch.Size([150050])
Testing data shape: torch.Size([50023, 47]) torch.Size([50023])
     actual  predicted
0     408.0  477.67099
1     410.0  477.67099
2     406.5  477.67099
3     410.5  477.67099
4     413.5  477.67099
..      ...        ...
123   460.0  477.67099
124   440.0  477.67099
125   432.0  477.67099
126   432.5  477.67099
127   423.5  477.67099

[128 rows x 2 columns]
Epoch 1/1000, Validation Loss: 1.3423


KeyboardInterrupt: 

In [ ]:
train.columns

Index(['tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'WIFI', 'BLE', 'rssi',
       'channel_rssi', 'channel_index', 'spreading_factor', 'bandwidth',
       'f_cnt', 'isHoliday', 'isExamTime', 'weekday', 'month', 'hour_sin',
       'semester_SS23', 'semester_WS22/23', 'semester_WS23/24', 'tmp_lag_1',
       'tmp_lag_2', 'tmp_lag_3', 'tmp_lag_4', 'tmp_lag_5', 'hum_lag_1',
       'hum_lag_2', 'hum_lag_3', 'hum_lag_4', 'hum_lag_5', 'CO2_lag_1',
       'CO2_lag_2', 'CO2_lag_3', 'CO2_lag_4', 'CO2_lag_5', 'CO2_next',
       'VOC_lag_1', 'VOC_lag_2', 'VOC_lag_3', 'VOC_lag_4', 'VOC_lag_5',
       'vis_lag_1', 'vis_lag_2', 'vis_lag_3', 'vis_lag_4', 'vis_lag_5',
       'CO2_next_scaled'],
      dtype='object')

In [50]:
from datetime import datetime

importlib.reload(utils)
start_time = datetime.strptime('2022-11-03 10:00:00', '%Y-%m-%d %H:%M:%S')
predictions = utils.predict_data_multivariate_transformer(start_time=start_time, selected_room='am001', prediction_count=5)
predictions

loading latest scaler: models/transformer_multivariate_quarter_hour_CO2_scaler_v1.pth
loading latest model: models/transformer_multivariate_quarter_hour_CO2_model_v1.pth


/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid f

,date_time_rounded,CO2_pred
0,2022-11-03 00:00:00,458.0
1,2022-11-03 00:15:00,457.0
2,2022-11-03 00:30:00,463.0
3,2022-11-03 00:45:00,462.0
4,2022-11-03 01:00:00,459.0
...,...,...
91,2022-11-03 22:45:00,461.0
92,2022-11-03 23:00:00,461.0
93,2022-11-03 23:15:00,461.0
94,2022-11-03 23:30:00,461.0


In [21]:
predictions.columns

Index(['date_time_rounded', 'tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'WIFI',
       'BLE', 'rssi', 'channel_rssi', 'channel_index', 'spreading_factor',
       'bandwidth', 'f_cnt', 'isHoliday', 'isExamTime', 'weekday', 'month',
       'hour_sin', 'semester_SS23', 'semester_WS22/23', 'semester_WS23/24',
       'group', 'device_id_hka-aqm-am001', 'device_id_hka-aqm-am002',
       'device_id_hka-aqm-am003a', 'device_id_hka-aqm-am003b',
       'device_id_hka-aqm-am004', 'device_id_hka-aqm-am005',
       'device_id_hka-aqm-am107', 'device_id_hka-aqm-am109',
       'device_id_hka-aqm-am110', 'device_id_hka-aqm-am111',
       'device_id_hka-aqm-am115', 'device_id_hka-aqm-am116',
       'device_id_hka-aqm-am117', 'device_id_hka-aqm-am123',
       'device_id_hka-aqm-am124', 'device_id_hka-aqm-am126',
       'device_id_hka-aqm-am201a', 'device_id_hka-aqm-am201b',
       'device_id_hka-aqm-am204', 'device_id_hka-aqm-am205',
       'device_id_hka-aqm-am209', 'device_id_hka-aqm-am210',
       'devi